In [29]:
from pathlib import Path
import pickle
from pathlib import Path
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
from matplotlib.lines import Line2D
import seaborn as sns
sns.set_theme()
import math
import pandas as pd
import random
import numpy as np
import sys
import shutil

import IMP
import IMP.atom

sys.path.append(str(Path(Path.home(), "Documents/xray/src")))
from refine import refine_hs_to_max_ff

In [32]:
data_dir = Path(Path.home(), "Documents/xray/dev/29_synthetic_native_3/data")
for i in range(2):
    for j in range(10):
        pdb_file = Path(data_dir, "cifs/7mhf_20/{}/{}.cif".format(i, j))
        print(pdb_file)

        new_pdb_dir = Path(data_dir, "cifs/7mhf_20_new/{}".format(j))
        new_pdb_dir.mkdir(parents=True, exist_ok=True)

        new_pdb_file = Path(new_pdb_dir, "{}.cif".format(i))
        shutil.copy(pdb_file, new_pdb_file)


/Users/matthew/Documents/xray/dev/29_synthetic_native_3/data/cifs/7mhf_20/0/0.cif
/Users/matthew/Documents/xray/dev/29_synthetic_native_3/data/cifs/7mhf_20/0/1.cif
/Users/matthew/Documents/xray/dev/29_synthetic_native_3/data/cifs/7mhf_20/0/2.cif
/Users/matthew/Documents/xray/dev/29_synthetic_native_3/data/cifs/7mhf_20/0/3.cif
/Users/matthew/Documents/xray/dev/29_synthetic_native_3/data/cifs/7mhf_20/0/4.cif
/Users/matthew/Documents/xray/dev/29_synthetic_native_3/data/cifs/7mhf_20/0/5.cif
/Users/matthew/Documents/xray/dev/29_synthetic_native_3/data/cifs/7mhf_20/0/6.cif
/Users/matthew/Documents/xray/dev/29_synthetic_native_3/data/cifs/7mhf_20/0/7.cif
/Users/matthew/Documents/xray/dev/29_synthetic_native_3/data/cifs/7mhf_20/0/8.cif
/Users/matthew/Documents/xray/dev/29_synthetic_native_3/data/cifs/7mhf_20/0/9.cif
/Users/matthew/Documents/xray/dev/29_synthetic_native_3/data/cifs/7mhf_20/1/0.cif
/Users/matthew/Documents/xray/dev/29_synthetic_native_3/data/cifs/7mhf_20/1/1.cif
/Users/matthew/D